# Lab - Rental Price 
Reference: Dataset was extracted from one of the popular property listing site (updated to Sep 2018)

Exercise: 
1. Try train on all states dataset, then predict the rental based on selected features
2. Try train on only Selangor dataset, then predict the rental based on selected features


In [209]:
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil

## Read dataset

In [315]:
df_orig = pd.read_csv("./data/RentIndexRaw.csv", low_memory=False)
COLUMNS = list(df_orig)
print (COLUMNS)

['ID', 'BuiltUpSize', 'City', 'CreatedDateTime', 'CreatedUser', 'DataSourceID', 'Furnishing', 'Latitude', 'Longitude', 'NoOfBathroom', 'NoOfBedroom', 'NoOfParking', 'Postcode', 'PostedDate', 'PropertyAddress', 'PropertyName', 'PropertyType', 'RentalPerMth', 'SourceUrl', 'State', 'UpdatedDateTime', 'UpdatedUser']


## Preparing dataset 

In [316]:
print('Original dataset: {}'.format(df_orig.shape))

df_orig['State'] = df_orig['State'].str.upper()
df_orig['City'] = df_orig['City'].str.upper()
df_orig['PropertyType'] = df_orig['PropertyType'].str.upper()

# Selangor
df_sel = df_orig[(df_orig.State == 'SELANGOR')]
print('Selangor dataset: {}'.format(df_sel.shape))

# KL
df_kul = df_orig[(df_orig.State == 'KUALA-LUMPUR')]
print('KL dataset: {}'.format(df_kul.shape))


# remove rows with zero value
df = df_sel[(df_sel[['BuiltUpSize','RentalPerMth', 'NoOfBedroom']] != 0).all(axis=1)]
                            #'NoOfBathroom', 'NoOfParking']] != 0).all(axis=1)]


print('Wanted dataset: {}'.format(df.shape))

Original dataset: (54405, 22)
Selangor dataset: (30292, 22)
KL dataset: (13648, 22)
Wanted dataset: (20626, 22)


## Split into dataset of: Train, Validation, Test (70%-20%-10%)

In [317]:

df_train0, df_valid0, df_test0 = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])

# Retrieve only Featured columns
df_train = df_train0[['State','BuiltUpSize','RentalPerMth','NoOfBedroom','Furnishing','PropertyType']]
df_valid = df_valid0[['State','BuiltUpSize','RentalPerMth','NoOfBedroom','Furnishing','PropertyType']]
df_test = df_test0[['State','BuiltUpSize','RentalPerMth','NoOfBedroom','Furnishing','PropertyType']]

print('Train: {}'.format(df_train.shape))
print('Valid: {}'.format(df_valid.shape))
print('Test: {}'.format(df_test.shape))

Train: (14438, 6)
Valid: (4125, 6)
Test: (2063, 6)


## Categorize string value columns

In [318]:
df_train['PropertyType'].unique()

array(['SERVICED RESIDENCE', 'CONDOMINIUM', 'APARTMENT',
       '2-STY TERRACE/LINK HOUSE', 'SEMI-DETACHED HOUSE', 'FLAT',
       '2.5-STY TERRACE/LINK HOUSE', '3-STY TERRACE/LINK HOUSE',
       '1-STY TERRACE/LINK HOUSE', 'BUNGALOW', 'TOWNHOUSE',
       '1.5-STY TERRACE/LINK HOUSE', 'CLUSTER HOUSE',
       '3.5-STY TERRACE/LINK HOUSE', '4-STY TERRACE/LINK HOUSE'],
      dtype=object)

In [319]:
df_train['State'].unique()

array(['SELANGOR'], dtype=object)

## Determine metrics for validation - RMSE

In [320]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    #queue_capacity = 100
      
  ))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
    
#print_rmse(model, df_valid)

## Model: DNNRegressor 

In [327]:
# DNNRegressor
LABEL = 'RentalPerMth'
OUTDIR = 'rentalDNNRegression_trained'
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# Categorize string features
categorical_column1 = tf.feature_column.categorical_column_with_vocabulary_list(
    key="State", vocabulary_list=['SELANGOR', 'JOHOR', 'KUALA-LUMPUR', 'PUTRAJAYA'], 
    default_value=0)

categorical_column2 = tf.feature_column.categorical_column_with_vocabulary_list(
    key="Furnishing", vocabulary_list=['Unfurnished', 'Fully Furnished', 'Partly Furnished', 'Unknown'], 
    default_value=0)

categorical_column3 = tf.feature_column.categorical_column_with_vocabulary_list(
    key="PropertyType", vocabulary_list=['CONDOMINIUM', 'SERVICED RESIDENCE', 'APARTMENT',
       '2-STY TERRACE/LINK HOUSE', '3-STY TERRACE/LINK HOUSE',
       '1-STY TERRACE/LINK HOUSE', 'TOWNHOUSE', 'SEMI-DETACHED HOUSE',
       'BUNGALOW', '1.5-STY TERRACE/LINK HOUSE',
       '3.5-STY TERRACE/LINK HOUSE', 'CLUSTER HOUSE', 'FLAT',
       '2.5-STY TERRACE/LINK HOUSE', '4-STY TERRACE/LINK HOUSE',
       'RESIDENTIAL LAND'], 
    default_value=0)


model = tf.estimator.DNNRegressor(    
    hidden_units = [64, 2048, 64],
    feature_columns = [tf.feature_column.numeric_column('BuiltUpSize'),
                       tf.feature_column.numeric_column('NoOfBedroom'),
                       #tf.feature_column.indicator_column(categorical_column1),
                       tf.feature_column.indicator_column(categorical_column2),
                       tf.feature_column.indicator_column(categorical_column3)
                      ], 
    activation_fn = tf.nn.relu,
    optimizer=tf.train.AdamOptimizer(
        learning_rate=0.001,
        #l1_regularization_strength=0.01
    ),

    model_dir = OUTDIR
)
 

model.train(input_fn = tf.estimator.inputs.pandas_input_fn(
    x = df_train,
    y = df_train[LABEL],
    batch_size = 128,
    num_epochs = 5,
    shuffle = True,
  ));

print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_master': '', '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb30ee2e10>, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_device_fn': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_model_dir': 'rentalDNNRegression_trained', '_session_config': None, '_num_worker_replicas': 1, '_task_type': 'worker', '_train_distribute': None, '_task_id': 0, '_is_chief': True, '_tf_random_seed': None, '_evaluation_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into rentalDNNRegression_trained/m

## Prediction 

In [328]:
predictions = model.predict(input_fn = tf.estimator.inputs.pandas_input_fn(
#    x = pd.DataFrame({
#                      'BuiltUpSize':[560,1200,1600],
#                      'NoOfBedroom':[2,3,3],
#                      'PropertyType':['CONDOMINIUM', 'SERVICED RESIDENCE', 'APARTMENT'],
#                      'Furnishing':['Unfurnished', 'Fully Furnished', 'Partly Furnished']}),
    x = df_test,
    y = None,
    batch_size = 128,
    shuffle = False
  ))

for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from rentalDNNRegression_trained/model.ckpt-564
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([1312.496], dtype=float32)}
{'predictions': array([2350.7822], dtype=float32)}
{'predictions': array([1934.3997], dtype=float32)}
{'predictions': array([4282.42], dtype=float32)}
{'predictions': array([2273.1992], dtype=float32)}
{'predictions': array([3787.0547], dtype=float32)}
{'predictions': array([3191.3438], dtype=float32)}
{'predictions': array([1694.0834], dtype=float32)}
{'predictions': array([1661.2302], dtype=float32)}
{'predictions': array([3168.1958], dtype=float32)}
{'predictions': array([2205.745], dtype=float32)}
{'predictions': array([2648.3079], dtype=float32)}
{'predictions': array([1336.4155], dtype=float32)}
{'predictions': array([1615.724], dtype=float32)}
{'predictions

{'predictions': array([1400.3445], dtype=float32)}
{'predictions': array([1964.7443], dtype=float32)}
{'predictions': array([1333.375], dtype=float32)}
{'predictions': array([1085.4052], dtype=float32)}
{'predictions': array([1593.0177], dtype=float32)}
{'predictions': array([1825.7006], dtype=float32)}
{'predictions': array([1517.338], dtype=float32)}
{'predictions': array([178.52222], dtype=float32)}
{'predictions': array([1625.3724], dtype=float32)}
{'predictions': array([2128.0137], dtype=float32)}
{'predictions': array([2445.239], dtype=float32)}
{'predictions': array([1903.0256], dtype=float32)}
{'predictions': array([1856.7753], dtype=float32)}
{'predictions': array([1342.0587], dtype=float32)}
{'predictions': array([3293.4902], dtype=float32)}
{'predictions': array([1203.5052], dtype=float32)}
{'predictions': array([2128.0137], dtype=float32)}
{'predictions': array([1533.7902], dtype=float32)}
{'predictions': array([1662.4691], dtype=float32)}
{'predictions': array([1666.9092],

{'predictions': array([1155.0654], dtype=float32)}
{'predictions': array([932.045], dtype=float32)}
{'predictions': array([1275.3588], dtype=float32)}
{'predictions': array([1297.2826], dtype=float32)}
{'predictions': array([507.5823], dtype=float32)}
{'predictions': array([1515.6927], dtype=float32)}
{'predictions': array([1713.4197], dtype=float32)}
{'predictions': array([927.05817], dtype=float32)}
{'predictions': array([2128.0137], dtype=float32)}
{'predictions': array([1408.3984], dtype=float32)}
{'predictions': array([927.05817], dtype=float32)}
{'predictions': array([1017.5961], dtype=float32)}
{'predictions': array([2547.8987], dtype=float32)}
{'predictions': array([1465.0433], dtype=float32)}
{'predictions': array([1319.5746], dtype=float32)}
{'predictions': array([1431.4315], dtype=float32)}
{'predictions': array([1100.2633], dtype=float32)}
{'predictions': array([3622.5332], dtype=float32)}
{'predictions': array([1698.6637], dtype=float32)}
{'predictions': array([2306.3616],

{'predictions': array([1817.29], dtype=float32)}
{'predictions': array([1007.0174], dtype=float32)}
{'predictions': array([1080.8583], dtype=float32)}
{'predictions': array([754.3619], dtype=float32)}
{'predictions': array([1016.3063], dtype=float32)}
{'predictions': array([1237.3258], dtype=float32)}
{'predictions': array([1604.8353], dtype=float32)}
{'predictions': array([1378.2533], dtype=float32)}
{'predictions': array([1408.7539], dtype=float32)}
{'predictions': array([1057.0813], dtype=float32)}
{'predictions': array([1342.0587], dtype=float32)}
{'predictions': array([1474.9147], dtype=float32)}
{'predictions': array([1519.2842], dtype=float32)}
{'predictions': array([2733.7568], dtype=float32)}
{'predictions': array([1078.168], dtype=float32)}
{'predictions': array([1251.6229], dtype=float32)}
{'predictions': array([1001.144], dtype=float32)}
{'predictions': array([966.59454], dtype=float32)}
{'predictions': array([2284.008], dtype=float32)}
{'predictions': array([1088.0394], dt

{'predictions': array([1813.3334], dtype=float32)}
{'predictions': array([1616.8094], dtype=float32)}
{'predictions': array([1217.9601], dtype=float32)}
{'predictions': array([2717.6653], dtype=float32)}
{'predictions': array([2136.2397], dtype=float32)}
{'predictions': array([1138.4913], dtype=float32)}
{'predictions': array([2458.401], dtype=float32)}
{'predictions': array([1940.1581], dtype=float32)}
{'predictions': array([3192.9893], dtype=float32)}
{'predictions': array([1730.3293], dtype=float32)}
{'predictions': array([1140.9872], dtype=float32)}
{'predictions': array([1626.6808], dtype=float32)}
{'predictions': array([1621.2875], dtype=float32)}
{'predictions': array([1027.4163], dtype=float32)}
{'predictions': array([1425.0226], dtype=float32)}
{'predictions': array([1405.5656], dtype=float32)}
{'predictions': array([1318.0841], dtype=float32)}
{'predictions': array([1002.0817], dtype=float32)}
{'predictions': array([1616.8094], dtype=float32)}
{'predictions': array([762.58795

{'predictions': array([1924.9447], dtype=float32)}
{'predictions': array([1286.6821], dtype=float32)}
{'predictions': array([1662.4178], dtype=float32)}
{'predictions': array([1773.9915], dtype=float32)}
{'predictions': array([1662.8243], dtype=float32)}
{'predictions': array([1921.9176], dtype=float32)}
{'predictions': array([1438.6688], dtype=float32)}
{'predictions': array([2635.4043], dtype=float32)}
{'predictions': array([1084.9988], dtype=float32)}
{'predictions': array([2195.874], dtype=float32)}
{'predictions': array([1078.168], dtype=float32)}
{'predictions': array([1166.0206], dtype=float32)}
{'predictions': array([1413.3341], dtype=float32)}
{'predictions': array([1293.2335], dtype=float32)}
{'predictions': array([1729.1642], dtype=float32)}
{'predictions': array([1401.8469], dtype=float32)}
{'predictions': array([4413.83], dtype=float32)}
{'predictions': array([1392.1184], dtype=float32)}
{'predictions': array([1083.4047], dtype=float32)}
{'predictions': array([3750.8599], 

{'predictions': array([2065.4956], dtype=float32)}
{'predictions': array([2187.648], dtype=float32)}
{'predictions': array([3788.5564], dtype=float32)}
{'predictions': array([1385.3655], dtype=float32)}
{'predictions': array([1226.4362], dtype=float32)}
{'predictions': array([1033.3408], dtype=float32)}
{'predictions': array([2084.5308], dtype=float32)}
{'predictions': array([1688.4403], dtype=float32)}
{'predictions': array([1691.1812], dtype=float32)}
{'predictions': array([2635.4043], dtype=float32)}
{'predictions': array([1698.6637], dtype=float32)}
{'predictions': array([1660.8239], dtype=float32)}
{'predictions': array([1622.1332], dtype=float32)}
{'predictions': array([1923.2994], dtype=float32)}
{'predictions': array([1515.6927], dtype=float32)}
{'predictions': array([1318.0841], dtype=float32)}
{'predictions': array([1607.6409], dtype=float32)}
{'predictions': array([2053.979], dtype=float32)}
{'predictions': array([1765.5809], dtype=float32)}
{'predictions': array([1592.0801]

{'predictions': array([1401.8469], dtype=float32)}
{'predictions': array([1297.2826], dtype=float32)}
{'predictions': array([1101.4509], dtype=float32)}
{'predictions': array([1695.3224], dtype=float32)}
{'predictions': array([1687.0961], dtype=float32)}
{'predictions': array([1858.6047], dtype=float32)}
{'predictions': array([1531.9613], dtype=float32)}
{'predictions': array([1167.2595], dtype=float32)}
{'predictions': array([2158.0342], dtype=float32)}
{'predictions': array([1620.0487], dtype=float32)}
{'predictions': array([1333.375], dtype=float32)}
{'predictions': array([2388.626], dtype=float32)}
{'predictions': array([1002.78925], dtype=float32)}
{'predictions': array([1887.1561], dtype=float32)}
{'predictions': array([1687.0961], dtype=float32)}
{'predictions': array([1877.64], dtype=float32)}
{'predictions': array([2366.5186], dtype=float32)}
{'predictions': array([1799.9086], dtype=float32)}
{'predictions': array([1762.526], dtype=float32)}
{'predictions': array([1991.4609], 

In [329]:
df_test.head(12)

,State,BuiltUpSize,RentalPerMth,NoOfBedroom,Furnishing,PropertyType
34273,SELANGOR,790,2300,1,Fully Furnished,CONDOMINIUM
17565,SELANGOR,1427,900,4,Partly Furnished,2-STY TERRACE/LINK HOUSE
18809,SELANGOR,1173,1100,4,Unfurnished,2-STY TERRACE/LINK HOUSE
38620,SELANGOR,2600,1500,4,Partly Furnished,SEMI-DETACHED HOUSE
41858,SELANGOR,1372,2300,3,Fully Furnished,CONDOMINIUM
35734,SELANGOR,2300,1800,4,Partly Furnished,2-STY TERRACE/LINK HOUSE
19205,SELANGOR,1937,1300,4,Unfurnished,2-STY TERRACE/LINK HOUSE
39838,SELANGOR,1020,1800,3,Fully Furnished,CONDOMINIUM
20737,SELANGOR,1001,1500,2,Fully Furnished,CONDOMINIUM
43062,SELANGOR,1916,4900,3,Fully Furnished,CONDOMINIUM
